In [45]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [46]:
data_train = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
data_test = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
data_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


Separate between categorical columns and numerical columns

In [47]:
# Paso 1. Copia para evitar SettingWithCopyWarning
data_train_clean = data_train.copy()
data_test_clean = data_test.copy()

# Paso 2. Dropear columnas con más de 20% NaN
def drop_20_null(df):
    null_percent = df.isna().mean() * 100
    drop_cols = null_percent[null_percent > 20].index.tolist()
    return df.drop(columns=drop_cols, axis=1)

data_train_clean = drop_20_null(data_train_clean)

# Paso 3. Definir features y target
y = data_train_clean['SalePrice']
X = data_train_clean.drop(columns=['SalePrice'])

# Paso 4. Detectar columnas categóricas y numéricas de forma coherente
cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()


Imputation process if needed

In [48]:
both_cols = cat_cols + num_cols

data_train_clean[both_cols].info() #seems there are few columns with NaN values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 74 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSZoning       1460 non-null   object 
 1   Street         1460 non-null   object 
 2   LotShape       1460 non-null   object 
 3   LandContour    1460 non-null   object 
 4   Utilities      1460 non-null   object 
 5   LotConfig      1460 non-null   object 
 6   LandSlope      1460 non-null   object 
 7   Neighborhood   1460 non-null   object 
 8   Condition1     1460 non-null   object 
 9   Condition2     1460 non-null   object 
 10  BldgType       1460 non-null   object 
 11  HouseStyle     1460 non-null   object 
 12  RoofStyle      1460 non-null   object 
 13  RoofMatl       1460 non-null   object 
 14  Exterior1st    1460 non-null   object 
 15  Exterior2nd    1460 non-null   object 
 16  ExterQual      1460 non-null   object 
 17  ExterCond      1460 non-null   object 
 18  Foundati

In [49]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1459 non-null   int64  
 1   MSSubClass     1459 non-null   int64  
 2   MSZoning       1455 non-null   object 
 3   LotFrontage    1232 non-null   float64
 4   LotArea        1459 non-null   int64  
 5   Street         1459 non-null   object 
 6   Alley          107 non-null    object 
 7   LotShape       1459 non-null   object 
 8   LandContour    1459 non-null   object 
 9   Utilities      1457 non-null   object 
 10  LotConfig      1459 non-null   object 
 11  LandSlope      1459 non-null   object 
 12  Neighborhood   1459 non-null   object 
 13  Condition1     1459 non-null   object 
 14  Condition2     1459 non-null   object 
 15  BldgType       1459 non-null   object 
 16  HouseStyle     1459 non-null   object 
 17  OverallQual    1459 non-null   int64  
 18  OverallC

In [50]:
data_test.shape

(1459, 80)

In [51]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# --- CONFIGURACIÓN DE IMPUTADORES ---
impute1 = SimpleImputer(strategy='mean')
impute2 = SimpleImputer(strategy='median')
num_imputers = [impute1, impute2]  # puedes probar ambos

# --- COLUMNAS NUMÉRICAS Y CATEGÓRICAS ---
cat_cols = data_train.select_dtypes(include=['object', 'category']).columns.tolist()
num_cols = data_train.select_dtypes(include=['int64', 'float64']).drop(columns='SalePrice').columns.tolist()

# --- FUNCIÓN PARA PIPELINE ---
def run_xgb_pipeline(X, y, num_cols, cat_cols, X_valid=None, y_valid=None, imputer=num_imputers[0]):
    """
    Entrena un pipeline de XGB con ColumnTransformer para numéricas y categóricas.
    - Si se pasan X_valid, y_valid, usa early stopping.
    - Devuelve el pipeline entrenado.
    """
    
    # Transformadores
    numerical_transformer = imputer
    categorical_transformer = Pipeline(steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.float64))
    ])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, num_cols),
            ('cat', categorical_transformer, cat_cols)
        ]
    )
    
    # Modelo
    model = XGBRegressor(
        random_state=0,
        learning_rate=0.05,
        n_estimators=1000
    )
    
    if X_valid is not None and y_valid is not None:
        # --- Transformamos train y valid manualmente ---
        preprocessor.fit(X)
        X_train_trans = preprocessor.transform(X)
        X_valid_trans = preprocessor.transform(X_valid)
        
        # Entrenamos modelo con early stopping
        model.fit(
            X_train_trans, y,
            eval_set=[(X_valid_trans, y_valid)],
            early_stopping_rounds=10,
            verbose=False
        )
        
        # Reconstruimos el pipeline ya entrenado
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        
        # Evaluación
        preds_valid = pipeline.predict(X_valid)
        print('MAE:', mean_absolute_error(y_valid, preds_valid))
        print('R2:', r2_score(y_valid, preds_valid))
        print('RMSE:', np.sqrt(mean_squared_error(y_valid, preds_valid)))
        
    else:
        # --- Entrenamiento con todo ---
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('model', model)
        ])
        pipeline.fit(X, y)
        print("Pipeline entrenado con todo el conjunto de entrenamiento")
    
    return pipeline

# --- USO DE LA FUNCIÓN CON SPLIT PARA EVALUACIÓN ---
X = data_train.drop(columns='SalePrice', axis=1)
y = data_train['SalePrice']

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, train_size=0.8, random_state=0, shuffle=True
)

pipeline_trained = run_xgb_pipeline(X_train, y_train, num_cols, cat_cols, X_valid, y_valid)

# --- USO DE LA FUNCIÓN PARA SUBMIT ---
pipeline_final = run_xgb_pipeline(X, y, num_cols, cat_cols)  # entrenamos con todo el train set

# --- PREDECIR TEST ---
X_test_final = data_test[X.columns]  # manteniendo las mismas columnas
preds_test = pipeline_final.predict(X_test_final)

# --- CREAR CSV SUBMISSION ---
output = pd.DataFrame({
    'Id': data_test['Id'],  # siempre usar la columna Id original
    'SalePrice': preds_test
})
output.to_csv('submission.csv', index=False)
print("CSV de submission generado correctamente")


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAE: 16988.863388270547
R2: 0.8388439109253172
RMSE: 33360.45221419148
Pipeline entrenado con todo el conjunto de entrenamiento
CSV de submission generado correctamente


In [52]:
run_xgb_pipeline(X, y, num_cols, cat_cols)

Pipeline entrenado con todo el conjunto de entrenamiento


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['Id', 'MSSubClass',
                                                   'LotFrontage', 'LotArea',
                                                   'OverallQual', 'OverallCond',
                                                   'YearBuilt', 'YearRemodAdd',
                                                   'MasVnrArea', 'BsmtFinSF1',
                                                   'BsmtFinSF2', 'BsmtUnfSF',
                                                   'TotalBsmtSF', '1stFlrSF',
                                                   '2ndFlrSF', 'LowQualFinSF',
                                                   'GrLivArea', 'BsmtFullBath',
                                                   'BsmtHalfBath', 'FullBath',
                                                   'HalfBath'...
                              feature_types=None, gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=0.05,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=1000, n_jobs=None,
                              num_parallel_tree=None, random_state=0, ...))])

It seems that the median imputation is better for this case.

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

def grid_search_cv(X, y):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                       train_size=0.8, 
                                                       random_state=0,
                                                       shuffle=True)

    numerical_transformer = SimpleImputer(strategy='mean')

    categorical_transformer = Pipeline(steps=[
        ('impute', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('numercial', numerical_transformer, num_cols),
            ('categorical', categorical_transformer, cat_cols)
        ])
    
    param_grid={
        'model__n_estimators':[100, 300],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split':[2, 5, 10],
        'model__min_samples_leaf':[1, 2, 4],
        'model__max_features':[1.0, 'sqrt']
    }

    scoring = ['neg_mean_absolute_error', 'r2', 'neg_root_mean_squared_error']
    
    rf = RandomForestRegressor(random_state=0)
    
    my_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', rf)
    ])


    grid_search = GridSearchCV(estimator=my_pipeline, param_grid=param_grid,
                              cv=5, scoring=scoring, refit='r2', n_jobs=-1)
    
    grid_search.fit(X_train, y_train)

    preds_valid = grid_search.predict(X_valid)

    print(f'Mejores Parametros:', grid_search.best_params_)
    print('R2 en validación:', r2_score(y_valid, preds_valid))
    print('MAE en validación:', mean_absolute_error(y_valid, preds_valid))

    return grid_search
    #preds_test = grid_search.predict(data_test)
    

In [58]:
grid_search_cv(X, y)

Mejores Parametros: {'model__max_depth': None, 'model__max_features': 1.0, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 300}
R2 en validación: 0.8383666251495696
MAE en validación: 17469.94720319635


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('numercial',
                                                                         SimpleImputer(),
                                                                         ['Id',
                                                                          'MSSubClass',
                                                                          'LotFrontage',
                                                                          'LotArea',
                                                                          'OverallQual',
                                                                          'OverallCond',
                                                                          'YearBuilt',
                                                                          'YearRemodAdd',
                                                                          'MasVnrArea',
                                                                          'BsmtFinSF1',
                                                                          'BsmtFinSF2',
                                                                          'BsmtUnfSF',
                                                                          'TotalBsmtSF',
                                                                          '1stFlrSF',
                                                                          '2ndFlrSF',
                                                                          'LowQualFinSF',
                                                                          'GrLivArea',
                                                                          'BsmtFullBath',
                                                                          'B...
                                                                          'CentralAir',
                                                                          'Electrical', ...])])),
                                       ('model',
                                        RandomForestRegressor(random_state=0))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [None, 10, 20],
                         'model__max_features': [1.0, 'sqrt'],
                         'model__min_samples_leaf': [1, 2, 4],
                         'model__min_samples_split': [2, 5, 10],
                         'model__n_estimators': [100, 300]},
             refit='r2',
             scoring=['neg_mean_absolute_error', 'r2',
                      'neg_root_mean_squared_error'])

In [59]:
# Preparar X_test con las mismas columnas que X
X_test_final = data_test_clean[X.columns]
print(len(X_test_final), len(data_test_clean))

1459 1459


In [60]:
# Entrenar GridSearch y obtener el mejor modelo
grid_model = grid_search_cv(X, y)

# Preparar X_test con las mismas columnas que X
X_test_final = data_test_clean[X.columns]

# Predecir
preds_test = grid_model.predict(X_test_final)

# Crear CSV para submit
output = pd.DataFrame({
    'Id': data_test_clean.index,
    'SalePrice': preds_test
})
output.to_csv('submission.csv', index=False)


Mejores Parametros: {'model__max_depth': None, 'model__max_features': 1.0, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 300}
R2 en validación: 0.8383666251495696
MAE en validación: 17469.94720319635
